In [1]:
%gui qt5

In [2]:
from lib.ui import spectrum_widget_Qt5 as swidget
import numpy as np
import pyqtgraph as pg
from lib.parsers import cameca
import PyQt5
from scipy import signal

## Constructing main window

In [3]:
# pg.setConfigOptions(useOpenGL=True)  # this option makes it worse on 2nd generation i5-core with integrated gpu

In [3]:
main_widget = PyQt5.QtWidgets.QWidget()
main_widget.setWindowTitle('WDS Scan Viewer')
main_layout = PyQt5.QtGui.QGridLayout(main_widget)
main_widget.setLayout(main_layout)

sample_list = PyQt5.QtWidgets.QListView()
sample_list.setMaximumWidth(150)
main_layout.addWidget(sample_list, 0, 0)
alpha_spin = PyQt5.QtWidgets.QSpinBox()
alpha_spin.setMaximum(255)
main_layout.addWidget(alpha_spin, 1, 0)
alpha_spin.setToolTip('alpha of curves (0 to 255)')
alpha_spin.setValue(175)
sub_widget = PyQt5.QtWidgets.QWidget()
main_layout.addWidget(sub_widget, 0, 1, 2, 1)

spec_widget = swidget.EDSSpectraGUI(initial_mode='wds', pet_opacity=0.91)
spec_widget.canvas.setLimits(xMax=200000)

spec_layout= PyQt5.QtGui.QHBoxLayout(sub_widget)
sub_widget.setLayout(spec_layout)
spec_layout.addWidget(spec_widget, 0)
spec_widget.setParent(sub_widget)

main_widget.show()

## Parsing and loading data:

In [4]:
cam_wds3 = cameca.CamecaWDS('../qbhm_datasets/spectra/MEGA_Full15kV.wdsDat')

In [5]:
model = cameca.CamecaWDSListModel(cam_wds3)
sample_list.setModel(model)
selection_model = sample_list.selectionModel()
sample_list.setSelectionMode(PyQt5.Qt.QAbstractItemView.ExtendedSelection)

## Customized functions for WDS GUI

In [6]:
items = {}
selection_model = sample_list.selectionModel()

def highlighter(selected, deselected):
    for i in selected.indexes():
        highlight_spectra(model.data(i, 0))
    for j in deselected.indexes():
        dehighlight_spectra(model.data(j, 0))

def set_wds(canvas, datasets, spect=0, sav_gol=False):
    items.clear()
    try:
        selection_model.selectionChanged.disconnect(highlighter)
    except TypeError:
        print('signal from selection model not connected')
    try:
        model.dataChanged.disconnect(change_visibility)
    except TypeError:
        print('signal from model not connected')
    try:
        alpha_spin.valueChanged.disconnect(change_global_alpha)
    except TypeError:
        print('signal from alpha spin box not connected')
        
    for i in datasets:
        index = datasets.index(i)
        x = np.arange(i.items[spect]['wds_start_pos'],
                      i.items[spect]['wds_start_pos'] + i.items[spect]['steps'] * i.items[spect]['step_size'],
                      i.items[spect]['step_size'],)
        y = i.items[spect]['data']
        if isinstance(sav_gol, tuple):
            y = signal.savgol_filter(y, sav_gol[0], sav_gol[1])
        itm = pg.PlotCurveItem(x, y, name=i.comment, pen=pg.mkPen(index))
        itm.setClickable(True, width=2)
        itm.sigClicked.connect(print_curve)
        #itm.setToolTip(i.comment)
        #itm.setAcceptHoverEvents(True)
        canvas.addItem(itm)
        if model.data(model.index(index), 10) == 0:
            itm.hide()
        items[i.comment] = itm

    canvas.set_xtal(i.items[spect]['2D'], i.items[spect]['K'])
    selection_model.selectionChanged.connect(highlighter)
    model.dataChanged.connect(change_visibility)
    alpha_spin.valueChanged.connect(change_global_alpha)
    return items

def change_global_alpha():
    alpha_val = alpha_spin.value()
    for i in items.values():
        color = i.opts['pen'].color()
        color.setAlpha(alpha_val)
        i.setPen(color)

def change_visibility(index):
    name = model.data(index, 0)
    state = model.data(index, 10)
    if state == 2:
        items[name].show()
    elif state == 0:
        items[name].hide()

def print_curve(itm):
    tw.setText(itm.name())
    
def highlight_spectra(label):
    if label in items:
        #items[label]._old_pen = items[label].opts['pen']
        color = items[label].opts['pen'].color()
        color.setAlpha(125)
        items[label].setShadowPen(color, width=8)
        items[label].setZValue(100.)
        color.setAlpha(255)
        items[label].setPen(color)
    
def dehighlight_spectra(label):
    if label in items:
        #items[label].setPen(items[label]._old_pen)
        items[label].setShadowPen(None)
        items[label].setZValue(0.0)
        alpha_val = alpha_spin.value()
        color = items[label].opts['pen'].color()
        color.setAlpha(alpha_val)
        items[label].setPen(color)

In [7]:
list_stuff = set_wds(spec_widget.canvas,
                     cam_wds3.datasets,
                     spect=1,
                     sav_gol=(7,3))


signal from selection model not connected
signal from model not connected
signal from alpha spin box not connected


In [14]:
spec_widget.canvas.clear()

In [16]:
main_widget.show()